# <h1 style="color:#1abc9c;">QwenPhysics: Fine-Tuning de QWEn2.5-3B con Artículos de Física de arXiv</h1>

## Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi

Wed May 14 07:03:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install required dependencies 
!pip3 install torch==2.4.0 torchvision==0.19.0 unsloth[cu124-torch240]==2025.2.4 --extra-index-url git+https://github.com/unslothai/unsloth.git
!pip3 install unsloth_zoo==2025.2.3 peft==0.14.0
!pip3 install datasets==3.2.0 bitsandbytes==0.45.2
!pip3 install trl==0.14.0 transformers==4.48.3
!pip3 install pyopenssl==25.0.0 accelerate==1.3.0

Looking in indexes: https://pypi.org/simple, git+https://github.com/unslothai/unsloth.git
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 k

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.15.2
    Uninstalling peft-0.15.2:
      Successfully uninstalled peft-0.15.2
  Attempting uninstall: unsloth_zoo
    Found existing installation: unsloth_zoo 2025.5.4
    Uninstalling unsloth_zoo-2025.5.4:
      Successfully uninstalled unsloth_zoo-2025.5.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.6.0
    Uninstalling datasets-3.6

In [ ]:
# Import necessary libraries for model training and dataset preparation
from unsloth import FastLanguageModel

# For dataset
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# For training
from trl import SFTTrainer, SFTConfig
from unsloth import is_bfloat16_supported

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Load Model

In [ ]:
# Define the repository and model name for loading the pre-trained model
repo_name = "Qwen" 
model_name = "Qwen2.5-Coder-3B-Instruct"  

# Load the pre-trained language model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=f"{repo_name}/{model_name}", 
    max_seq_length=4096,  # Set the maximum sequence length for tokenization
    dtype=None,  # Define the data type (e.g., float16, bfloat16) or auto-detect
    load_in_4bit=True  # Enable 4-bit quantization to reduce memory usage
)

==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Apply Lora

In [ ]:
# Apply LoRA (Low-Rank Adaptation) to the model for efficient fine-tuning.
# LoRA reduces the number of trainable parameters, making fine-tuning
# more memory-efficient while preserving model performance.
model = FastLanguageModel.get_peft_model(
    model,

    # LoRA rank: Determines the number of learnable parameters per layer.
    # Higher values increase expressiveness but also memory usage.
    r=32,  # Common values: 8, 16, 32, 64, 128

    # List of model layers to which LoRA will be applied.
    # These layers are typically key components in transformer-based models.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],

    # Scaling factor for LoRA. It influences the learning rate adaptation.
    lora_alpha=16,

    # Dropout rate for LoRA layers. Setting it to 0 is optimized for stability.
    lora_dropout=0.1,  # A non-zero value can improve generalization in some cases.

    # Whether to train bias parameters. "none" is optimized for efficiency.
    bias="none",

    # Enable gradient checkpointing to reduce memory usage for long context models.
    # "unsloth" is a specialized version optimized for Unsloth models.
    use_gradient_checkpointing="unsloth",

    # Whether to use Rank-Stabilized LoRA (rslora), which adapts LoRA dynamically.
    use_rslora=True,

    # Configuration for LoftQ (Low-rank Quantization), which reduces model size.
    # Setting it to None disables LoftQ.
    loftq_config=None,

    # Select specific layers to apply LoRA to. Helps control memory usage and efficiency.
    # In this case, LoRA is applied from layer 7 to 24.
    layers_to_transform=list(range(7, 24, 1))
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.4 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


## Prepare dataset

In [ ]:
# Prepare dataset by formatting prompts for training
tokenizer = get_chat_template(tokenizer, chat_template="chatml")

def formatting_prompts_func(examples):

    prompt = examples["question"]
    output = examples["answer"]

    messages = []
    for p, o in zip(prompt, output):
        el = [
              {"role": "system", "content": "You are an expert physics assistant."},
              {"role": "user", "content": p},
              {"role": "assistant", "content": o},
            ]
        messages.append(el)

    texts = [tokenizer.apply_chat_template(
        ele, tokenize=False, add_generation_prompt=False) for ele in messages]
    return {"text": texts, }


dataset = load_dataset("gallen881/arxiv-physics", split="train")
dataset = dataset.select(range(8000))
# Other datasets 
#dataset = load_dataset("HydraLM/physics_dataset_alpaca", split="train")
# dataset = load_dataset("camel-ai/physics", split="train")

dataset_p = dataset.map(formatting_prompts_func, batched=True)
ds = dataset_p.train_test_split(test_size=0.2)

train_ds = ds["train"]
test_ds = ds["test"]

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

(…)-00000-of-00001-5bba4a271402bdbb.parquet:   0%|          | 0.00/11.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30231 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
train_ds[:1]

{'question': ['Which instruments are used to study black hole star?'],
 'answer': ["Black holes cannot be directly observed as they do not emit light. Therefore, instruments are used to observe the effects of their gravity on nearby matter. Here are some of the instruments used to study black holes:\n\n1. X-ray telescopes - These telescopes can detect the high-energy X-rays emitted by matter as it falls into a black hole.\n\n2. Radio telescopes - They can observe the jets of particles that shoot out from black holes as matter is sucked into them.\n\n3. Infrared telescopes - They can detect the heat generated by the accretion of matter into a black hole.\n\n4. Gravitational wave detectors - These instruments detect the ripple in spacetime caused by the merging of black holes.\n\n5. Optical telescopes - While they can't directly observe black holes, these telescopes can detect the effects of a black hole's gravity on nearby stars by measuring their orbits."],
 'text': ["<|im_start|>syste

## Train the model

In [ ]:
# Configure the training process using SFTTrainer (Supervised Fine-Tuning Trainer)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,

    train_dataset=train_ds, 
    eval_dataset=test_ds, 

    # Training arguments
    args=SFTConfig(
        dataset_text_field="text",
        max_seq_length=4094, 

        dataset_num_proc=8,  # Number of CPU processes for dataset preprocessing
        packing=False,  # Whether to concatenate multiple examples into a single sequence

        # Training batch size per GPU/TPU/CPU
        per_device_train_batch_size=20,

        # Number of steps to accumulate gradients before performing a backward pass
        gradient_accumulation_steps=4,

        # Number of training epochs
        num_train_epochs=1,
        # Number of steps is other option
        # max_steps=200,

        # Number of warmup steps for the learning rate scheduler
        warmup_steps=20,

        # Learning rate for the optimizer
        learning_rate=2e-4,

        # Use 16-bit floating-point precision (FP16) if BFloat16 is not supported
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),  # Use BFloat16 if the hardware supports it

        # Frequency of logging training progress
        logging_steps=10,

        # Optimizer type; "adamw_8bit" reduces memory usage
        optim="adamw_8bit",

        # Weight decay for regularization (helps prevent overfitting)
        weight_decay=0.01,

        # Type of learning rate scheduler (linear decay in this case)
        lr_scheduler_type="linear",

        # Directory to save training outputs (e.g., checkpoints, logs)
        output_dir=f"outputs/train/Physics-{model_name}-BTs",

        # Disable reporting to external loggers (e.g., WandB, TensorBoard)
        report_to="none",

        # Evaluation settings
        fp16_full_eval=True,  # Use FP16 precision during evaluation
        per_device_eval_batch_size=8,  # Batch size per device for evaluation
        eval_accumulation_steps=4,  # Accumulate gradients during evaluation

        # Evaluation strategy (run evaluation every `eval_steps`)
        eval_strategy="steps",
        eval_steps=20,  
    ),
)

Map (num_proc=8):   0%|          | 0/6400 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1600 [00:00<?, ? examples/s]

In [ ]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,400 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 20 | Gradient Accumulation steps = 4
\        /    Total batch size = 80 | Total steps = 80
 "-____-"     Number of trainable parameters = 28,270,592


Step,Training Loss,Validation Loss
20,1.229800,1.110521
40,1.056000,1.048512
60,1.033200,1.037643
80,1.016900,1.034365


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


## Save the model

In [ ]:
# Save the quantized fine-tuned model for later use
model.save_pretrained_gguf(f"outputs/gguf/{model_name}-BTs", tokenizer, quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.74 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 36/36 [00:01<00:00, 21.02it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving outputs/gguf/Qwen2.5-Coder-3B-Instruct-BTs/pytorch_model-00001-of-00002.bin...
Unsloth: Saving outputs/gguf/Qwen2.5-Coder-3B-Instruct-BTs/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at outputs/gguf/Qwen2.5-Coder-3B-Instruct-BTs into f16 GGUF format.
The output location will be /content/outputs/gguf/Qwen2.5-Coder-3B-Instruct-BTs/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Qwen2.5-Coder-3B-Instruct-BTs
INFO:hf-to-gguf:Model architecture: Qwen2ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO

In [ ]:
# Save the LoRA model for later use
model.save_pretrained_merged(f"outputs/loras/{model_name}-BTs", tokenizer, save_method="lora")

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model...

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

 Done.


In [ ]:
# Download and install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp -b b4710
!cd llama.cpp && cmake -B build && cmake --build build --config Release

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found Op

In [ ]:
# Convert the LoRA to llama.cpp
!python3 llama.cpp/convert_lora_to_gguf.py outputs/loras/Qwen2.5-Coder-3B-Instruct-BTs

INFO:lora-to-gguf:Loading base model from Hugging Face: unsloth/qwen2.5-coder-3b-instruct-bnb-4bit
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:lora-to-gguf:Exporting model...
INFO:hf-to-gguf:blk.10.ffn_down.weight.lora_a, torch.float32 --> F32, shape = {11008, 32}
INFO:hf-to-gguf:blk.10.ffn_down.weight.lora_b, torch.float32 --> F32, shape = {32, 2048}
INFO:hf-to-gguf:blk.10.ffn_gate.weight.lora_a, torch.float32 --> F32, shape = {2048, 32}
INFO:hf-to-gguf:blk.10.ffn_gate.weight.lora_b, torch.float32 --> F32, shape = {32, 11008}
INFO:hf-to-gguf:blk.10.ffn_up.weight.lora_a, torch.float32 --> F32, shape = {2048, 32}
INFO:hf-to-gguf:blk.10.ffn_up.weight.lora_b, torch.float32 --> F32, shape = {32, 11008}
INFO:hf-to-gguf:blk.10.attn_k.weight.lora_a, torch.float32 --> F32, shape = {2048, 32}
INFO:hf-to-gguf:blk.10.attn_k.weight.lora_b, torch.float32 --> F32, shape = {32, 256}
INFO:hf-to-gguf:blk.10.attn_output.weight.lora_a, torch.float32 --> F32, shape = {2048, 3

## Load the model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install gradio==5.16.0
!pip3 install huggingface_hub==0.28.1
!pip3 install llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
!pip3 install langchain==0.3.17 langchain-core==0.3.33 langchain-community==0.3.14
!pip3 install pydantic==2.9.2

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124


In [ ]:
# Import necessary libraries
from langchain_community.chat_models import ChatLlamaCpp
import gradio as gr

In [ ]:
# Create the LLM
# Although it's slower, we load the full model instead of just the LoRA adapters
# This approach simplifies deployment and provides better compatibility with llama.cpp
llm = ChatLlamaCpp(
    model_path="/content/drive/MyDrive/unsloth-F16.gguf",
    n_gpu_layers=25,
    stop=["<|im_end|>\n"],
    n_ctx=4096,
    max_tokens=4096,
    streaming=True,
    n_batch=256,
    temperature=0.1,
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_load_model_from_file: using device CUDA0 (Tesla T4) - 14992 MiB free
llama_model_loader: loaded meta data with 26 key-value pairs and 434 tensors from /content/drive/MyDrive/unsloth-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 Coder 3b Instruct Bnb 4bit
llama_model_loader: - kv   3:                       general.organization str              = Unsloth
llama_model_loader: - kv   4:                           general.finetune st

## Test the model with chatbot

In [5]:
# Create the llama handler
def llama_cpp(message, history):

    messages = history + [
        {
            "role": "user",
            "metadata": None,
            "content": message,
            "options": None,
        }
    ]

    with open('messages.txt', 'w') as f:
      f.write(str(messages))
    response = ""

    for c in llm.stream(messages):
        response += c.content
        yield response

In [6]:
# Create and launch the web chatbot
demo = gr.ChatInterface(
    llama_cpp,
    type="messages",
    flagging_mode="manual",
    flagging_options=[],
    save_history=True,
)

if __name__ == "__main__":
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://99267a49bb8e8b095c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Riemann zeta function?  
When was the michelson morley experiment?    
Can you write the expresion for riemann zeta function?
What is the term used to describe black holes?
preguntar a ver si sabe integrar funciones impares en un intervalo simétrico